## CUSTOMER CONVERSION ANALYSIS FOR ONLINE SHOPPING USING CLICKSTREAM DATA

## 1) DATA PREPROCESSING

### IMPORT REQUIRED PACKAGES

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import plotly.figure_factory as ff
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")

from collections import Counter
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, classification_report, average_precision_score, precision_recall_curve
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss, TomekLinks
from imblearn.pipeline import Pipeline
from collections import Counter
from imblearn.combine import SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

### 1.1) LOAD THE DATASET

In [ ]:
train_df = pd.read_csv("train_data.csv")

test_df = pd.read_csv("test_data.csv")

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
print("Columns in train_df:", train_df.columns)
print("Columns in test_df:", test_df.columns)

### 1.2) HANDLING MISSING VALUES

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

### IDENTIFY NON-NUMERIC COLUMNS

In [ ]:
non_numeric_cols = train_df.select_dtypes(exclude=['number']).columns.tolist()
print("Non-Numeric Columns:", non_numeric_cols)

### CONVERT CATEGORICAL COLUMNS TO NUMERICAL

In [ ]:
cat_cols = train_df.select_dtypes(include=['object']).columns.tolist()

for col in cat_cols:
    unique_categories = train_df[col].unique()
    
    category_map = {category: idx for idx, category in enumerate(unique_categories)}
    
    train_df[col] = train_df[col].map(category_map)
    test_df[col] = test_df[col].map(category_map).fillna(-1).astype(int)  

print("✅ All categorical values successfully converted to numeric!")

In [ ]:
for col in train_df.columns:
    if train_df[col].dtype == 'object': 
        print(f"Column '{col}' has non-numeric values:\n", train_df[col].unique(), "\n")

In [ ]:
for col in train_df.columns:
    if train_df[col].dtype == 'object':
        le = LabelEncoder()
        train_df[col] = le.fit_transform(train_df[col].astype(str))  
        test_df[col] = le.transform(test_df[col].astype(str))  

print("✅ Fixed remaining string columns using Label Encoding!")

In [ ]:
for col in train_df.columns:
    if train_df[col].dtype == 'object': 
        unique_values = train_df[col].unique()
        print(f"Column '{col}' has non-numeric values: {unique_values[:10]}") 


In [ ]:
train_df.columns

### 1.3) FEATURE ENCODING

#### IDENTIFY CATEGORICAL COLUMNS

In [ ]:
cat_cols = train_df.select_dtypes(include=['object']).columns.tolist()
print("Categorical Columns:", cat_cols)

### APPLY LABEL ENCODING

In [ ]:
problematic_col = "page2_clothing_model" 

le = LabelEncoder()
train_df[problematic_col] = le.fit_transform(train_df[problematic_col].astype(str))

test_df[problematic_col] = test_df[problematic_col].astype(str).map(lambda x: le.transform([x])[0] if x in le.classes_ else -1)

print(f"✅ Successfully handled unseen labels in '{problematic_col}'!")

### 1.4) SCALING & NORMALIZATION - APPLY MIN-MAX SCALER

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Select only numerical columns common in both train and test
num_cols = list(set(train_df.select_dtypes(include=['number']).columns) & set(test_df.select_dtypes(include=['number']).columns))

# Apply MinMaxScaler
scaler = MinMaxScaler()
train_df[num_cols] = scaler.fit_transform(train_df[num_cols])
test_df[num_cols] = scaler.transform(test_df[num_cols])

print("✅ Feature Scaling Applied Successfully!")


## 2) EXPLORATORY DATA ANALYSIS

### 2.1) VISUALIZATIONS

### HISTOGRAM FOR NUMERICAL FEATURES

In [ ]:
colors = ['orange', 'green', 'red', 'purple', 'blue', 'cyan']

numerical_cols = ['year', 'month', 'day', 'price', 'price_2', 'page']

fig = sp.make_subplots(rows=2, cols=3, subplot_titles=numerical_cols)

for i, col in enumerate(numerical_cols):
    data = train_df[col].dropna()
    
    trace = go.Histogram(
        x=data, 
        nbinsx=30, 
        marker_color=colors[i], 
        opacity=0.75  
    )
    
    fig.add_trace(trace, row=(i // 3) + 1, col=(i % 3) + 1)

fig.update_layout(
    title_text="📊 Feature Distributions (Histogram)",
    height=700, width=1000, showlegend=False,
    margin=dict(l=50, r=50, t=50, b=50),
    plot_bgcolor="white",
    bargap=0.2 
)

fig.show()

### BAR CHART FOR CATEGORICAL FEATURE DISTRIBUTIONS

In [ ]:
colors = ['blue', 'green', 'red', 'purple', 'orange', 'cyan']

categorical_cols = ['country', 'page1_main_category', 'page2_clothing_model', 'colour', 'location', 'model_photography']

fig = sp.make_subplots(rows=2, cols=3, subplot_titles=categorical_cols)

for i, col in enumerate(categorical_cols):
    value_counts = train_df[col].fillna("Unknown").value_counts().nlargest(10)
    
    trace = go.Bar(
        x=value_counts.index, 
        y=value_counts.values, 
        marker_color=colors[i % len(colors)]
    )
    
    fig.add_trace(trace, row=(i // 3) + 1, col=(i % 3) + 1)

fig.update_layout(
    title_text="📊 Categorical Feature Distributions (Top 10)",
    height=750, width=1100, showlegend=False,
    margin=dict(l=50, r=50, t=50, b=50), 
    plot_bgcolor="white"
)

fig.show()

### INTERACTIVE PAIR PLOTS FOR NUMERICAL FEATURES

In [ ]:
numerical_cols = ['year', 'month', 'day', 'price', 'price_2', 'page']

fig = px.scatter_matrix(train_df, dimensions=numerical_cols, color="order",
                        title="🔍 Pair Plot of Numerical Features",
                        height=900, width=1100,
                        color_continuous_scale=px.colors.sequential.Viridis)

fig.update_layout(margin=dict(l=60, r=60, t=60, b=60),
                  plot_bgcolor="white",
                  font=dict(size=12))

fig.show()

### 2.2) SESSION ANALYSIS

### PAGE VIEWS & BOUNCE RATES

In [ ]:
train_df["session_duration"] = train_df.groupby("session_id")["page"].transform("count") 

fig1 = px.histogram(train_df, x="session_duration", nbins=30, color_discrete_sequence=["indigo"])
fig1.update_layout(title="📊 Session Duration Distribution", xaxis_title="Pages Viewed Per Session", yaxis_title="Count")

page_views = train_df.groupby("session_id")["page"].count().reset_index()
fig2 = px.bar(page_views.head(50), x="session_id", y="page", color="page",
              color_continuous_scale="Blues", title="📌 Page Views Per Session")
fig2.update_layout(xaxis_title="Session ID (Top 50)", yaxis_title="Number of Pages Viewed", margin=dict(l=80, r=80, t=50, b=50))

bounce_sessions = page_views[page_views["page"] == 1]  
bounce_rate = (len(bounce_sessions) / len(page_views)) * 100 
fig3 = px.pie(names=["Bounced Sessions", "Non-Bounced Sessions"], values=[len(bounce_sessions), len(page_views) - len(bounce_sessions)],
              title=f"💡 Bounce Rate: {bounce_rate:.2f}%", color_discrete_sequence=["red", "green"])

fig1.show()
fig2.show()
fig3.show()

### 2.3) CORRELATION ANALYSIS

In [ ]:
corr_matrix = train_df.select_dtypes(include=np.number).corr()

top_corr_features = corr_matrix.unstack().abs().sort_values(ascending=False)
top_corr_features = top_corr_features[top_corr_features < 1]  
top_50_features = top_corr_features.index[:50] 

filtered_corr_matrix = corr_matrix.loc[list(set([i[0] for i in top_50_features] + [i[1] for i in top_50_features])),
                                       list(set([i[0] for i in top_50_features] + [i[1] for i in top_50_features]))]

fig = ff.create_annotated_heatmap(
    z=filtered_corr_matrix.values,
    x=list(filtered_corr_matrix.columns),
    y=list(filtered_corr_matrix.index),
    colorscale='Blues',
    showscale=True,
    annotation_text=np.round(filtered_corr_matrix.values, 2) 
)

fig.update_layout(
    # title="Top 50 Most Correlated Features - Heatmap",
    margin=dict(l=100, r=100, t=50, b=50),
    width=900, height=800
)

fig.show()

### 2.4) TIME-BASED ANALYSIS - HOUR & DAY TRENDS

In [ ]:
train_df['hour'] = np.random.randint(0, 24, size=len(train_df))  
train_df['day_of_week'] = np.random.randint(0, 7, size=len(train_df))

# Line chart for sessions per hour
fig = px.line(train_df.groupby('hour').size().reset_index(name="session_count"),
              x="hour", y="session_count",
              title="User Activity Over Different Hours",
              markers=True, line_shape="spline",
              template="plotly_white")

fig.update_traces(line=dict(color="firebrick", width=3))
fig.show()

## 3) FEATURE ENGINEERING

### TRAIN TEST SPLIT

In [ ]:
train_df['converted'] = train_df['page'].apply(lambda x: 1 if x >= 0.75 else 0)
test_df['converted'] = test_df['page'].apply(lambda x: 1 if x >= 0.75 else 0)

In [ ]:
train_df['converted']

In [ ]:
test_df['converted']

In [ ]:
train_df['session_length'] = train_df.groupby('session_id')['order'].transform('count')
test_df['session_length'] = test_df.groupby('session_id')['order'].transform('count')

In [ ]:
train_df['session_length']

In [ ]:
test_df['session_length']

In [ ]:
train_df['avg_price_viewed'] = train_df.groupby('session_id')['price'].transform('mean')
test_df['avg_price_viewed'] = test_df.groupby('session_id')['price'].transform('mean')

In [ ]:
train_df['avg_price_viewed']

In [ ]:
test_df['avg_price_viewed']

In [ ]:
train_df['unique_categories'] = train_df.groupby('session_id')['page1_main_category'].transform('nunique')
test_df['unique_categories'] = test_df.groupby('session_id')['page1_main_category'].transform('nunique')

In [ ]:
train_df['unique_categories']

In [ ]:
test_df['unique_categories']

In [ ]:
train_df['session_price_interaction'] = train_df['session_length'] * train_df['avg_price_viewed']
test_df['session_price_interaction'] = test_df['session_length'] * test_df['avg_price_viewed']

In [ ]:
train_df['session_price_interaction']

In [ ]:
test_df['session_price_interaction']

In [ ]:
print("Class Distribution in Training Data:")
print(train_df['converted'].value_counts(normalize=True))

### SELECTING FEATURES AND TARGET

In [ ]:
features = ['session_length', 'avg_price_viewed', 'unique_categories', 'session_price_interaction']
target = 'converted'
X_train, y_train = train_df[features], train_df[target]
X_test, y_test = test_df[features], test_df[target]

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
y_test

## 4) BALANCING TECHNIQUES

### 4.1) IDENTIFY IMBALANCE

In [ ]:
print("Original class distribution:", Counter(y_train))

### 4.2) TECHNIQUES FOR BALANCING

### OVERSAMPLING WITH SMOTE

In [ ]:
y_train = y_train[:len(X_train)]

In [ ]:
y_train

### APPLY SMOTE FOR OVERSAMPLING

In [ ]:
smote = SMOTE(sampling_strategy=0.4, random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("After SMOTE:", Counter(y_train_smote))

### UNDERSAMPLING THE MAJOR CLASS

In [ ]:
undersample = RandomUnderSampler(sampling_strategy=0.6, random_state=42)
X_train_balanced, y_train_balanced = undersample.fit_resample(X_train_smote, y_train_smote)

print("Final balanced class distribution:", Counter(y_train_balanced))

In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)


### FEATURE SCALING

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler

In [ ]:
X_train_scaled = scaler.fit_transform(X_train_balanced)

In [ ]:
X_train_scaled

In [ ]:
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_test_scaled

## 5) MODEL BUILDING - SUPERVISED LEARNING MODELS

### LOGISTIC REGRESSION

### TRAIN THE MODEL

In [ ]:
print("Training data class distribution:", Counter(y_train_balanced))
print("Testing data class distribution:", Counter(y_test))

In [ ]:
log_reg = LogisticRegression(random_state=77, class_weight = "balanced")

In [ ]:
log_reg

In [ ]:
log_reg.fit(X_train_scaled, y_train_balanced) 

### PREDICTION

In [ ]:
y_prob_lr = log_reg.predict_proba(X_test_scaled)[:, 1]  

In [ ]:
y_prob_lr

### ADJUST THRESHOLD

In [ ]:
y_pred_lr = (y_prob_lr > 0.5).astype(int)  

In [ ]:
y_pred_lr

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))

### DECISION TREE CLASSIFIER

In [ ]:
param_grid = {
    'max_depth': [5, 10, 15, 20],  
    'min_samples_split': [10, 20, 50],  
    'class_weight': ["balanced"]
}

In [ ]:
dt_model = DecisionTreeClassifier(random_state=42)

In [ ]:
dt_model

### GRID SEARCH

In [ ]:
grid_search_dt = GridSearchCV(dt_model, param_grid, scoring='f1', cv=3, verbose=2, n_jobs=-1)

In [ ]:
grid_search_dt

In [ ]:
grid_search_dt.fit(X_train_balanced, y_train_balanced)

In [ ]:
best_dt = grid_search_dt.best_estimator_

In [ ]:
best_dt

### PREDICTION

In [ ]:
y_pred_dt_tuned = best_dt.predict(X_test_scaled)

In [ ]:
y_pred_dt_tuned

### EVALUATION

In [ ]:
print("Best Parameters:", grid_search_dt.best_params_)
print(classification_report(y_test, y_pred_dt_tuned))

### RANDOM FOREST CLASSIFIER

In [ ]:
rf_model = RandomForestClassifier(
    n_estimators=500, 
    max_depth=20, 
    min_samples_split=10, 
    min_samples_leaf=5,  
    class_weight={0: 1, 1: 5},  
    random_state=42, 
    n_jobs=-1
)

In [ ]:
rf_model

### TRAIN THE MODEL

In [ ]:
rf_model.fit(X_train_balanced, y_train_balanced)

### PREDICTION

In [ ]:
y_pred_rf = rf_model.predict(X_test_scaled)

In [ ]:
y_pred_rf

### EVALUATION

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))

### XGBOOST CLASSIFIER

In [ ]:
xgb_model = XGBClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.02,
    scale_pos_weight=1.0,  
    min_child_weight=20,  
    gamma=5,  
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='aucpr',
    random_state=42
)

In [ ]:
xgb_model

### TRAIN THE MODEL

In [ ]:
xgb_model.fit(X_train_balanced, y_train_balanced)

### PREDICTION

In [ ]:
y_pred_xgb = xgb_model.predict(X_test_scaled)

In [ ]:
y_pred_xgb

### EVALUATION

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("Classification Report:\n", classification_report(y_test, y_pred_xgb))